In [15]:
import ROOT as r
from array import array

print("Done")

Done


In [ ]:
# === Output file and tree ===
output_file = ROOT.TFile("test_duplicates.root", "RECREATE")
tree = ROOT.TTree("ST-b2cc", "Test Tree with duplicate events")

# === Buffers ===
bp_p_buf = array('f', [0])   # Float_t
event_buf = array('Q', [0])  # ULong64_t

tree.Branch("Bp_P", bp_p_buf, "Bp_P/F")
tree.Branch("event", event_buf, "event/l")

# === Data to insert ===
data = [
    # Event 1121654010
    (1121654010, [139387.8125, 135842.546875, 119004.640625, 111138.6875, 138274.65625]),

    # Event 19726357590
    (19726357590, [
        127264.421875, 121160.2578125, 127050.9453125, 120945.6796875,
        121916.0703125, 115807.109375, 121702.2890625, 115592.078125,
        122707.90625, 122491.2265625, 117349.453125, 117132.59375
    ])
]

# === Fill tree ===
for event_id, bp_p_list in data:
    for val in bp_p_list:
        event_buf[0] = event_id
        bp_p_buf[0] = val
        tree.Fill()

# === Write to file ===
tree.Write()
output_file.Close()

print("✅ test_duplicates.root written with 17 entries.")


In [24]:
#Check some entries in some branch

filename = "test_duplicates.root"
file = r.TFile.Open(filename)
tree = file.Get("ST-b2cc") #or st-b2oc

entries = range(0, 16)

for entry in entries:
    tree.GetEntry(entry)
    print(tree.Bp_P)

139387.8125
135842.546875
119004.640625
111138.6875
138274.65625
127264.421875
121160.2578125
127050.9453125
120945.6796875
121916.0703125
115807.109375
121702.2890625
115592.078125
122707.90625
122491.2265625
117349.453125


In [ ]:
import ROOT
from collections import defaultdict, Counter

# Open the ROOT file and get the tree
file = ROOT.TFile.Open('/Users/ary/Desktop/Ramon Data/Fitted_Data_Clean_Bp_P/2024/Real/B2CC/2024_B2CC_DOWN_B7.root')
tree = file.Get("ST-b2cc")

# FILLs of interest
fills_of_interest = {
    10197, 10199, 10200, 10201, 10204, 10206,
    10207, 10208, 10209, 10210, 10211, 10213
}

# Containers for event IDs per FILL
events_by_fill = defaultdict(list)

# Loop over entries
for entry in tree:
    fill = int(entry.FILL)
    if fill in fills_of_interest:
        evt = int(entry.event)
        events_by_fill[fill].append(evt)

# Analyze duplicates
print(f"{'FILL':>6} | {'Duplicates':>10} / {'Total':<10} | {'Fraction'}")
print("-" * 45)

for fill in sorted(events_by_fill):
    events = events_by_fill[fill]
    total = len(events)
    duplicates = sum(count > 1 for count in Counter(events).values())
    ratio = duplicates / total if total > 0 else 0
    print(f"{fill:>6} | {duplicates:>10} / {total:<10} | {ratio:.4f}")
